## Дообучение Saiga-Mistral, квантизация и инференс с помощью llama-cpp

В репозитории реализован код для дообучения русскоязычной LLM [Saiga mistral](https://huggingface.co/IlyaGusev/saiga_mistral_7b_lora), а также её квантизация и запуск с помощью llama-cpp. Попытался сделать код максимально гибким и воспроизводимым.  
Предполагается запуск на GPU. Может запускаться на multi-gpu без доп. модификаций.  
При создании ноутбука опирался на эту [статью](https://habr.com/ru/articles/776872/) на Хабре, задекорировал и актуализировал некоторые моменты

In [ ]:
!pip install peft

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import PeftModel, PeftConfig, AutoPeftModelForCausalLM
from datasets import load_dataset
import transformers
import torch
import time
import os

In [ ]:
!pip install bitsandbytes

In [2]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

st_time = time.time()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant=False,
)


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit = True,
    torch_dtype=torch.float16,
    device_map="auto",

)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    is_trainable = True,
    quantization=bnb_config
)

model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

print(generation_config)
print(f'Загрузка модели заняла {round(time.time() - st_time, 2)} секунд')

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9
}

Загрузка модели заняла 7.67 секунд


In [3]:
model.print_trainable_parameters()

trainable params: 16,777,216 || all params: 6,755,192,832 || trainable%: 0.2484


In [4]:
!nvidia-smi

Thu Jan  2 09:02:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P0             31W /   70W |    3195MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


### Загрузка датасета

Датасет для дообучения должен быть в формате json и иметь формат ```[{"system": str, "user": str, "bot": str}, ... ]```, где system - системное сообщение для модели (например, у Сайги в use-example это "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."), user - это промпт пользователя, bot - ответ модели.

In [9]:
import os
print(os.listdir("/kaggle/input"))


from os import walk
filenames = next(walk("/kaggle/input/fine-tunning-llama/"), (None, None, []))[2] 
filenames

['fine-tunning-llama']


['train.json', 'val.json']

In [ ]:
TRAIN_PATH = "train.json"
VALID_PATH = "val.json"

data = load_dataset(
    "json", 
    data_files={
                'train': TRAIN_PATH,
                'validation': VALID_PATH
    }
)
data["train"] = data["train"].shuffle() # for train data shuffling, optional

## Предобработка датасета

In [ ]:
CUTOFF_LEN = 2500 # до какого токена будет обрезать текст


def generate_prompt(data_point):
    prompt = f"""<s>system
{data_point['system']}</s><s>user
{data_point['user']}</s><s>bot
{data_point['bot']}[</s>"""
    return prompt
 
    
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
    result["labels"] = result["input_ids"].copy()
    return result


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt


In [ ]:
train_data = (
    data["train"].map(generate_and_tokenize_prompt)
)

val_data = (
    data["validation"].map(generate_and_tokenize_prompt)
)

## Обучение модели

In [ ]:
BATCH_SIZE = 6
MICRO_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_EPOCHS = 5
OUTPUT_DIR = "finetuned_model"

training_arguments = transformers.TrainingArguments(
            per_device_train_batch_size=MICRO_BATCH_SIZE,
            per_device_eval_batch_size=MICRO_BATCH_SIZE,
            prediction_loss_only=True,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
            num_train_epochs=TRAIN_EPOCHS,
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_steps=25000,
            optim="adamw_torch",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            output_dir=OUTPUT_DIR,
            load_best_model_at_end=True,
            report_to=None,
            overwrite_output_dir=True,
)

In [ ]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model = torch.compile(model)
trainer.train()
model.save_pretrained(OUTPUT_DIR)

## Квантизация модели

Для начала склонируем репозитории с библиотеками rulm и llama-cpp для конкатенации обученного адаптера и квантизации.

In [ ]:
!git clone https://github.com/IlyaGusev/rulm.git
!git clone https://github.com/ggerganov/llama.cpp

### Склеим модель и обученный адаптер

In [ ]:
from rulm.self_instruct.src.tools import convert_to_native

In [ ]:
PATH_TO_CHECKPOINT = "finetuned_model/checkpoint-66" # путь до чекпоинта адаптера, который хотим приклеить
MERGED_MODEL_PATH = "merged_model.pt"

convert_to_native.convert_to_native(PATH_TO_CHECKPOINT, MERGED_MODEL_PATH, 
                                    device="cuda", enable_offloading=True)

### Конвертируем склеенную модель в 16-битный формат GGUF для запуска с помощью llama-cpp

In [ ]:
# сперва сохраним токенайзер в папку, где лежит лучший чекпоинт

tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga2_7b_lora", use_fast=False)
tokenizer.save_pretrained(PATH_TO_CHECKPOINT)

Обязательная строчка, надо откатить версию llama-cpp, т.к. на последней квантизация почему-то не работает.

In [ ]:
%cd llama.cpp
!git checkout 64e64aa

In [ ]:
OUTPUT_PATH = "../model-f16.gguf"

In [ ]:
!python convert.py {os.path.join("..", MERGED_MODEL_PATH)} --vocab-dir {os.path.join("..", PATH_TO_CHECKPOINT)} --outfile {OUTPUT_PATH} --outtype f16 --ctx 4096

### Квантуем моедль в 4 бита и 8 бит

In [ ]:
!make quantize

In [ ]:
QUANT_MODEL = "../model-q4_0.gguf"
QUANTIZATION_TYPE = "q4_0" # "q4_0" или "q4_1"

In [ ]:
! ./quantize {OUTPUT_PATH} {QUANT_MODEL} {QUANTIZATION_TYPE}

### Запуск скомпилированной версии на GPU с помощью llama-cpp

Переустановим llama-cpp на последнюю версию. Параметры, которые идут перед установкой, обязательны для запуска на GPU.

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

### Использование модели в питоновском коде

In [ ]:
from llama_cpp import Llama

In [ ]:
llm = Llama(model_path="../model-q4_0.gguf", n_gpu_layers=128, n_ctx=2048)

In [ ]:
!nvidia-smi

In [ ]:
prompt = f"""<s>system
    {'Any system prompt'}</s><s>user
    {'Any user prompt'}</s><s>bot"""

In [ ]:
start_time = time.time()

output = llm(
      prompt, # Prompt
      max_tokens=2048,
      echo=False,
      temperature=0
)

print(time.time() - start_time)

In [ ]:
print(output["choices"][0]["text"][:-1])